In [ ]:
import sys
sys.path.append('../')
sys.path.append('../build')
sys.path.append('../../')

import Springtronics as spr
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
system = spr.MechanicalSystem()

In [ ]:
m = 2
b= 2
k= 6
force = 1

print(f'Eta {b/(2*np.sqrt(k*m))}')

In [ ]:
filelength = 78001
sr = 16000
x = np.linspace(0, filelength/sr, filelength)

In [ ]:
dofName = 'oscillator'
system.degreesOfFreedom[f'{dofName}'] = spr.ParametricVariable(m)
system.interactionPotentials[f'{dofName}_K'] =  spr.IntegerPotential(k)
system.interactionPotentials[f'{dofName}_K'].degreesOfFreedom[dofName] = 2
system.interactionPotentials[f'{dofName}_B'] =  spr.LocalDamping(dofName, b)

In [ ]:
m_cant = 10
b_cant = 10
k_cant = 5
gamma = 10
duffing  = 10000 #lambda

In [ ]:
dofName = 'cantilever'
system.degreesOfFreedom[f'{dofName}'] = spr.ParametricVariable(m_cant)
system.interactionPotentials[f'{dofName}_K'] =  spr.IntegerPotential(k_cant)
system.interactionPotentials[f'{dofName}_K'].degreesOfFreedom[dofName] = 2
#spring
system.interactionPotentials[f'{dofName}_L'] =  spr.IntegerPotential(duffing)
system.interactionPotentials[f'{dofName}_L'].degreesOfFreedom[dofName] = 4
#duffing potential (lambda)
system.interactionPotentials[f'{dofName}_B'] =  spr.LocalDamping(dofName, b_cant)

In [ ]:
opticalDOF = 'oscillator'
mechanicalDOF = 'cantilever'

In [ ]:
system.interactionPotentials[f'OptoCoup_{opticalDOF}_{mechanicalDOF}'] =  spr.IntegerPotential(-gamma)
system.interactionPotentials[f'OptoCoup_{opticalDOF}_{mechanicalDOF}'].degreesOfFreedom[opticalDOF] = 2
system.interactionPotentials[f'OptoCoup_{opticalDOF}_{mechanicalDOF}'].degreesOfFreedom[mechanicalDOF] = 1
#optomechanical coupling (gamma)

In [ ]:
system.excitationSources['soundData'] = spr.DirectCInjectionSource(force)


system.interactionPotentials[f'excitation'] = spr.Excitation(opticalDOF, 'soundData', 1.0)

In [ ]:
#probe string
system.probes['system_output'] = spr.WindowedA2Probe(opticalDOF,
                                                    startIndex=0,
                                                    endIndex=filelength)

# Define an adjoint source (used to compute the gradient efficiently: https://en.wikipedia.org/wiki/Adjoint_state_method)
system.interactionPotentials['adjoint_source'] = system.probes['system_output'].makeAdjointSource()

#probe cantilever
system.probes['cant_output'] = spr.WindowedA2Probe(mechanicalDOF,
                                                    startIndex=0,
                                                    endIndex=filelength)

# Define an adjoint source (used to compute the gradient efficiently: https://en.wikipedia.org/wiki/Adjoint_state_method)
system.interactionPotentials['adjoint_source'] = system.probes['cant_output'].makeAdjointSource()

In [ ]:
env = spr.CPPEnvironment(numSteps = filelength,
                           timeStep = 1.0/sr,
                           numSweepSteps = 1,
                           numThreads=1)

In [ ]:
import Springtronics.LinearSimulation as lin
lin.makeLinearizedModel(system, np.zeros(1000))

In [ ]:
import Springtronics.HelperFunctions as hf
hf.getGamma(system)

In [ ]:
%%time
traj = env.getTrajectories(system, initialConditions=np.zeros(2), deleteTemp=False)

# Plot

## String

In [ ]:
plt.plot(x, traj[:,0])
plt.title(f'Final energy: {round(traj[-1, -2], 8)}\nFinal amplitude: {round(traj[-1,0], 8)}')

## Cantilever

In [ ]:
plt.plot(x, traj[:,1])
plt.title(f'Final energy: {round(traj[-1, -1], 8)}\nFinal amplitude: {round(traj[-1,1], 8)}')